In [0]:
# %% [code]
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# %% [code]
import seaborn as sns
import warnings 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import sklearn.metrics as metrics
import math

# %% [code]
df_train = pd.read_csv(r'../input/train-house/Train.csv')
df_test = pd.read_csv(r'../input/test-house/Test.csv')

# %% [code]
## Select object data type columns. 
object_columns_train = df_train.select_dtypes(include=['object'])

## Select numeric data type columns.
numerical_columns_train =df_train.select_dtypes(exclude=['object'])

# %% [code]
## Fill missing values in society column with 'None'.
object_columns_train['society'] = object_columns_train['society'].fillna('None')

# %% [code]
## Fill missing values in size & location column with most frequent value.

cols = ['size', 'location']
object_columns_train[cols] = object_columns_train[cols].fillna(object_columns_train.mode().iloc[0])

# %% [code]
## FIll missing values in bath column with its median.

median = numerical_columns_train['bath'].median()
numerical_columns_train['bath'] = numerical_columns_train['bath'].fillna(median)

# %% [code]
## Fill missing values in balcony column with 0

numerical_columns_train['balcony'] = numerical_columns_train['balcony'].fillna(0)

# %% [code]
## Remove availability column from dataset due to low variance.

object_columns_train.drop(['availability'], axis=1, inplace=True)

# %% [code]
#Using One hot encoder on categorical variables 

object_columns_train = pd.get_dummies(object_columns_train, columns= object_columns_train.columns) 

# %% [code]
object_columns_train.head(3)

# %% [code]
## Concat Categorical (after encoding) and numerical features

df_final_train = pd.concat([object_columns_train, numerical_columns_train], axis=1,sort=False)
df_final_train.head()

# %% [code]
df_final_train.shape

# %% [code]
## Make copy of train & test dataset.
df_train_copy = df_train.copy()
df_test_copy = df_test.copy()

# %% [markdown]
# ### Prepare test dataset 

# %% [code]
## Drop price column which has to be predicted.

df_test_copy.drop(['price'], axis=1, inplace=True)

# %% [markdown]
# ## XGB Regressor

# %% [code]
##Separate Train and Targets

target= np.log(df_final_train['price'])

df_final_train.drop(['price'],axis=1, inplace=True)

# %% [code]
df_final_train.shape

# %% [code]
x_train,x_test,y_train,y_test = train_test_split(df_final_train, target, test_size=0.3,random_state=15)

# %% [code]
xgb =XGBRegressor( booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=4, min_child_weight=1.5, n_estimators=2000,
             n_jobs=1, nthread=None, objective='reg:linear',
             reg_alpha=0.6, reg_lambda=0.6, scale_pos_weight=1, 
             silent=None, subsample=0.8, verbosity=1, tree_method='gpu_hist')


# %% [code]
#Fitting
xgb.fit(x_train, y_train)

# %% [markdown]
# ## LGB Regressor

# %% [code]
df_train_lgb = df_final_train.copy()
target_lgb = target.copy()

# %% [code]
import re
df_train_lgb = df_train_lgb.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

# %% [code]
x_train1,x_test1,y_train1,y_test1 = train_test_split(df_train_lgb, target_lgb, test_size=0.3,random_state=15)

# %% [code]
lgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=10000,
                                       max_bin=180,
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.4, 
                                       device= 'gpu',
                                       gpu_platform_id= 0,
                                       gpu_device_id=0
                                       )

# %% [code]
#Fitting
lgbm.fit(x_train1, y_train1, eval_metric='rmse')

# %% [markdown]
# ### Prediction from XGB Regressor and LGB Regressor both.

# %% [code]
predict1 = xgb.predict(x_test)
predict = lgbm.predict(x_test)

# %% [code]
## Error check
print('Root Mean Square Error test = ' + str(math.sqrt(metrics.mean_squared_error(y_test, predict1))))
print('Root Mean Square Error test = ' + str(math.sqrt(metrics.mean_squared_error(y_test, predict))))

# %% [markdown]
# ### Fitting and predicting whole train dataset from XGB & LGB Regressor.

# %% [code]
## Fit with all dataset in XGB Regressor
xgb.fit(df_final_train, target)

# %% [code]
lgbm.fit(df_train_lgb, target_lgb, eval_metric='rmse')

# %% [markdown]
# ### Now, predict test dataset. 

# %% [code]
#predict4 = lgbm.predict(df_test_copy)
predict3 = xgb.predict(df_test_copy)

# %% [code]
